# F-тест: Совместная значимость
Тестируется гипотеза
$$H_0:\beta_1=\cdots=\beta_J=0 $$
Тестовая статистика (неробастная) $$F=\frac{R^2-R^2_{restr}}{1-R^2}\cdot\frac{n-k-1}{J}=\frac{RSS_{restr}-RSS}{RSS}\cdot\frac{n-k-1}{J}$$
$R^2_{restr}, RSS_{restr}$ – из промежуточной регрессии с ограничениями

Критическое значение $$F_{cr}=F_{df1=J, df2=n-k-1}(\alpha)$$

Гипотеза отвергается если $F>F_{cr}$ или $P<\alpha$

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col # вывод подгонки
from scipy.stats import f # F-распределение

In [5]:
sleep_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/sleep75.csv')
mod1 = smf.ols(formula='sleep~totwrk+age+male+south+smsa+yngkid+marr+union', data=sleep_df).fit()
summary_col(mod1, stars=True)

,sleep
Intercept,3446.8303***
,(81.8399)
totwrk,-0.1691***
,(0.0181)
age,2.7145*
,(1.4724)
male,87.1081**
,(35.1732)
south,102.2718**
,(41.9250)


Потестируем совместную значимость `smsa, yngkid, marr, union`, т.е. гипотезу
$$H_0:\beta_{smsa}=\beta_{yngkid}=\beta_{marr}=\beta_{union}=0$$
Критическое значений: df1=4, df2=n-k-1

In [6]:
f.ppf(q=1-0.05, dfn=4, dfd=mod1.df_resid)

2.3847110483450447

### Первый способ: через спецификацию гипотезы

In [7]:
print( mod1.f_test('smsa=yngkid=marr=union=0') )

<F test: F=0.9097649568071344, p=0.4576805992248939, df_denom=697, df_num=4>


In [9]:
print( mod1.wald_test('smsa=yngkid=marr=union=0', use_f=True) )

<F test: F=array([[0.90976496]]), p=0.4576805992248939, df_denom=697, df_num=4>


### Второй способ: через вспомогательную регрессию с ограничениями

In [10]:
mod1_restr = smf.ols(formula='sleep~totwrk+age+male+south', data=sleep_df).fit()
mod1.compare_f_test(mod1_restr)

(0.9097649568071163, 0.4576805992248939, 4.0)

# F-тест: Значимость регрессии

Тестируется гипотеза
$$H_0:\beta_1=\cdots=\beta_k=0 $$
Тестовая статистика (неробастная) $$F=\frac{R^2}{1-R^2}\cdot\frac{n-k-1}{k}$$
Критическое значение $$F_{cr}=F_{df1=k, df2=n-k-1}(\alpha)$$

Гипотеза отвергается если $F>F_{cr}$ или $P<\alpha$

In [11]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_params # вывод результатов тестирования
from scipy.stats import f # F-распределение

## Sleep equation 1
Для датасета `sleep75` рассмотрим регрессию **sleep на totwrk, age, age^2, south, male, smsa, yngkid, marr, union**
Подгонка модели

In [12]:
# загрузим данные
sleep_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/sleep75.csv')
# подгоним модель
mod1 = smf.ols(formula='sleep~totwrk+age+I(age**2)+south+male+smsa+yngkid+marr+union', data=sleep_df).fit()

Результаты F-теста на значимость регрессии (тестовая статистика и P-значение)

In [13]:
mod1.fvalue, mod1.f_pvalue

(11.803442151355272, 2.1244300478607532e-17)

5%-критическое значение F-распределения

In [14]:
f.ppf(q=1-0.05, dfn=mod1.df_model, dfd=mod1.df_resid)

1.8933165104204854

*Вывод*: регрессия значима!

# F-тест: Линейные ограничения
Тестируется гипотеза о линейных ограничения на коэффициенты (в матричной форме)
$$H_0: R\beta=q$$
Тестовая статистика (неробастная) $$F=\frac{1}{J}(R\hat{\beta}-q)^\top(R\hat{V}R^\top)^{-1} (R\hat{\beta}-q)\quad \hat{V}=s^2(X^\top X)^{-1}$$


Критическое значения ($J$ – число независимых ограничений) $$F_{cr}=F_{df1=J, df2=n-k-1}(\alpha)$$

Гипотеза отвергается если $F>F_{cr}$ или $P<\alpha$

In [16]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col # вывод подгонки
from scipy.stats import f # F-распределение

## Output equation
Для набора данных `Labour` рассмотрим линейную регрессию **log(output) на log(capital), log(labour), log(wage)**

Вектор коэффициентов для этой модели $$\beta=\begin{pmatrix} const & \beta_{capital} & \beta_{labour} & \beta_{wage} \end{pmatrix}^\top$$

Результаты подгонки

In [17]:
labour_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/Labour.csv')
mod = smf.ols(formula='np.log(output)~np.log(capital)+np.log(labour)+np.log(wage)', data=labour_df).fit()
mod.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         np.log(output)   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.888
No. Observations:                 569   F-statistic:                     1499.
Covariance Type:            nonrobust   Prob (F-statistic):          1.65e-268
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -5.0073      0.221    -22.649      0.000      -5.442      -4.573
np.log(capital)     0.1493      0.015     10.141      0.000       0.120       0.178
np.log(labour)      0.7204      0.019     37.487      0.000       0.683       0.758
np.log(wage)        0.9214      0.058     16.001      0.000       0.808       1.034
===================================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Гипотеза 1
Тестируем $H_0:\beta_{capital}+\beta_{labour}+\beta_{wage}=1$

In [18]:
print( mod.f_test('np.log(capital)+np.log(labour)+np.log(wage)=1') )

<F test: F=198.58693315849249, p=7.280959454661686e-39, df_denom=565, df_num=1>


In [19]:
# критическое значение
f.ppf(q=1-0.05, dfn=1, dfd=mod.df_resid)

3.8579698801476354

*Техничекие подробности*: Запишем ограничение в матричном виде $R\beta=q$ как систему линейных уравнений. Имеем
\begin{align}
    R&=\begin{pmatrix} 0 & 1 & 1& 1 \end{pmatrix}&  q&=1
\end{align}

In [20]:
 print( mod.f_test( r_matrix=([0, 1, 1, 1], 1) ) )

<F test: F=198.58693315849249, p=7.280959454661686e-39, df_denom=565, df_num=1>


### Гипотеза 2
Тестируем $H_0:\beta_{labour}=\beta_{wage}$ 

Альтернативная запись $H_0:\beta_{labour}-\beta_{wage}=0$

In [21]:
print( mod.f_test('np.log(labour)=np.log(wage)') )

<F test: F=11.08097824634225, p=0.0009289780461988149, df_denom=565, df_num=1>


In [22]:
# критическое значение
f.ppf(q=1-0.05, dfn=1, dfd=mod.df_resid)

3.8579698801476354

*Техничекие подробности*: Запишем ограничение в матричном виде $R\beta=q$ как систему линейных уравнений. Имеем
\begin{align}
    R&=\begin{pmatrix} 0 & 0 & 1 & -1 \end{pmatrix}&  q&=0
\end{align}

In [24]:
 print( mod.f_test( r_matrix=([0, 0, 1, -1], 0) ) )

<F test: F=11.08097824634225, p=0.0009289780461988149, df_denom=565, df_num=1>


# F-тест: Структурные изменения

In [25]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col # вывод подгонки
from scipy.stats import f # F-распределение
import seaborn as sns

## Sleep equation
Рассмотрим регрессию **sleep/60 на totwrk, age, age^2, south, smsa, marr**

* для М $sleep/60=\beta_0+\beta_1totwrk+\beta_2age+\beta_3age^2+\beta_4south+\beta_5smsa+\beta_6marr+u$
* для Ж $sleep/60=\gamma_0+\gamma_1totwrk+\gamma_2age+\gamma_3age^2+\gamma_4south+\gamma_5smsa+\gamma_6marr+v$

Будем тестировать $H_0:\beta_j=\gamma_j$ (всего $k+1$ ограничение)

Уровень значимости $\alpha=0.01$

Для тестовой статистики подговим модель
* только по М
* только по Ж
* по полному датасету

Результаты подгонки

In [26]:
sleep_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/sleep75.csv')
specification = 'I(sleep/60)~totwrk+age+I(age**2)+south+smsa+marr'
# подгонка по полному датасету
mod_overall = smf.ols(formula=specification, data=sleep_df).fit()
# подгонка только по М
mod_men_only = smf.ols(formula=specification, data=sleep_df[ sleep_df['male']==1 ]).fit()
# подгонка только по Ж
mod_women_only = smf.ols(formula=specification, data=sleep_df[ sleep_df['male']==0 ]).fit()
# Вывод трёх регрессий в одной таблице
summary_col(results=[mod_men_only, mod_women_only, mod_overall], stars=True, model_names=['муж', 'жен', 'Общая'],
           info_dict={'N': lambda x: x.nobs, 'F-stat': lambda x: x.fvalue})

,муж,жен,Общая
Intercept,58.6338***,65.5581***,60.4212***
,(4.8114),(5.8327),(3.6715)
totwrk,-0.0031***,-0.0023***,-0.0025***
,(0.0004),(0.0005),(0.0003)
age,0.1221,-0.4440,-0.1229
,(0.2366),(0.3061),(0.1875)
I(age ** 2),-0.0008,0.0057,0.0021
,(0.0028),(0.0037),(0.0022)
south,1.0267,2.1532**,1.5738**
,(0.9330),(1.0648),(0.6985)


In [27]:
# Ингредиенты тестовой статистики
print('RSS (overall)=', mod_overall.ssr)
print('RSS (men_only)=', mod_men_only.ssr)
print('RSS (women_only)=', mod_women_only.ssr)
print('n=', mod_overall.nobs)
print('k=', mod_overall.df_model)

RSS (overall)= 33859.796879006404
RSS (men_only)= 17618.97919500131
RSS (women_only)= 15717.914859424825
n= 706.0
k= 6.0


Тестовая статистика $$F=\frac{RSS_{overall}-RSS_{men}-RSS_{women}}{RSS_{men}+RSS_{women}}*\frac{n-2(k+1)}{k+1}$$
Критическое значение $$F_{cr}=F_{df1=k+1, df2=n-2(k+1)}(\alpha)$$

In [28]:
F = (mod_overall.ssr-mod_men_only.ssr-mod_women_only.ssr)/(mod_men_only.ssr+mod_women_only.ssr)*(mod_overall.nobs-2*(mod_overall.df_model+1))/(mod_overall.df_model+1)
F_cr = f.ppf(q=1-0.01, dfn=mod_overall.df_model+1, dfd=mod_overall.nobs-2*(mod_overall.df_model+1))
F, F_cr

(1.5506147376999553, 2.6651528022423494)

# t-тест в Python

In [29]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_params # вывод результатов тестирования
from scipy.stats import t # t-распределение

## Значимость коэффициентов
Тестируем гипотезу $H_0:\beta=0$

Тестовая статистика $$t=\frac{\hat{\beta}}{s.e.(\beta)}$$

Критическое $t_{cr}=t_{df=n-k-1}(\alpha)$

Для набора данных `sleep75` рассмотрим линейную регрессию
**sleep на totwrk, age, south, male, smsa, yngkid, marr, union, log(hrwage)**

In [30]:
# Загрузим данные
df_sleep = pd.read_csv('sleep75.csv')
mod1 = smf.ols(formula='sleep~totwrk+age+south+male+smsa+yngkid+marr+union+np.log(hrwage)', 
                       data=df_sleep).fit()
# коэффициенты
mod1.params.round(3)

Intercept         3431.804
totwrk              -0.158
age                  2.437
south               78.046
male                36.485
smsa               -34.965
yngkid              50.136
marr                54.072
union               27.019
np.log(hrwage)      -2.727
dtype: float64

In [31]:
# Число наблюдений в модели, число регрессоров и степени свободы для t_cr
mod1.nobs, mod1.df_model, mod1.df_resid

(532.0, 9.0, 522.0)

In [32]:
# 5%-критическое значение t-распределения
t.ppf(q=1-0.05/2, df=mod1.df_resid)

1.9645189418326978

### Значимость выбранных коэффициентов
Потестируем значимость $\beta_{totwrk}$ и $\beta_{male}$

In [33]:
mod1.t_test('totwrk=0, male=0')

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.1581      0.021     -7.576      0.000      -0.199      -0.117
c1            36.4854     43.350      0.842      0.400     -48.677     121.648

## Общий t-тест
Тестируем $H_0:\beta=\theta$ ($\theta$ - заданное число)

Тестовая статистика $$t=\frac{\hat{\beta}-\theta}{s.e.(\beta)}$$

Для набора данных `Labour` рассмотрим регрессию **log(output) на log(capital) и log(labour)**


In [34]:
# Загрузим данные
df_labour = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/Labour.csv')
# Результаты подгонки
mod2 = smf.ols(formula='np.log(output)~np.log(capital)+np.log(labour)', data=df_labour).fit()
summary_params(mod2)

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,-1.711459,0.096711,-17.696720,4.187931e-56,-1.901415,-1.521504
np.log(capital),0.207570,0.017188,12.076774,4.875689e-30,0.173811,0.241329
np.log(labour),0.714847,0.023142,30.890017,1.569331e-123,0.669393,0.760301


Тестируется гипотеза $H_0:\beta_{capital}=0.5$ 

Результаты тестирования

In [35]:
mod2.t_test('np.log(capital)=0.5')

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.2076      0.017    -17.014      0.000       0.174       0.241

In [36]:
# 1%-критическое значение t-распределения
t.ppf(q=1-0.01/2, df=mod2.df_resid)

2.584543428450176

# Доверительные интервалы
Пусть $\gamma$ – доверительная вероятность и $t_{cr}=t_{df=n-k-1}(\alpha=1-\gamma)$

Доверительный интервал для коэффициента $\beta$: 
$$\left(\hat{\beta}-t_{cr}\cdot s.e.(\beta)\, ,\, \hat{\beta}+t_{cr}\cdot s.e.(\beta)\right)$$

In [37]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col # вывод подгонки
from scipy.stats import t # t-распределение

## Output equation

Для набора данных `Labour` рассмотрим регрессию **log(output) на log(capital) и log(labour)**

Результаты подгонки

In [38]:
labour_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/Labour.csv')
mod = smf.ols(formula='np.log(output)~np.log(capital)+np.log(labour)', data=labour_df).fit()
mod.summary(slim=True)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         np.log(output)   R-squared:                       0.838
Model:                            OLS   Adj. R-squared:                  0.837
No. Observations:                 569   F-statistic:                     1462.
Covariance Type:            nonrobust   Prob (F-statistic):          2.64e-224
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.7115      0.097    -17.697      0.000      -1.901      -1.522
np.log(capital)     0.2076      0.017     12.077      0.000       0.174       0.241
np.log(labour)      0.7148      0.023     30.890      0.000       0.669       0.760
===================================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Построим 95%-доверительные интервалы для каждого коэффициента ($\gamma=0.95$, $\alpha=0.05$)

In [39]:
mod.conf_int(alpha=0.05)

,0,1
Intercept,-1.901415,-1.521504
np.log(capital),0.173811,0.241329
np.log(labour),0.669393,0.760301


In [40]:
# ручное построение c.i.
t_cr = t.ppf(q=1-0.05/2, df=mod.df_resid)
mod.params-mod.bse*t_cr, mod.params+mod.bse*t_cr

(Intercept         -1.901415
 np.log(capital)    0.173811
 np.log(labour)     0.669393
 dtype: float64,
 Intercept         -1.521504
 np.log(capital)    0.241329
 np.log(labour)     0.760301
 dtype: float64)

Построим 90%-доверительные интервалы для каждого коэффициента ($\gamma=0.90$, $\alpha=0.10$)

In [41]:
mod.conf_int(alpha=0.10)

,0,1
Intercept,-1.870795,-1.552124
np.log(capital),0.179253,0.235888
np.log(labour),0.676720,0.752974
